In [147]:
# !pip install webdriver-manager
# !pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pprint
import re
import pandas as pd
import json
from tqdm import tqdm
import random
import requests


In [ ]:
def get_catalogs_wb():
    """получение каталога вб"""
    url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    data = response.json()
    with open('wb_catalogs_data.json', 'w', encoding='UTF-8') as file:
        json.dump(data, file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в wb_catalogs_data_sample.json')
    data_list = []
    for d in data:
        try:
            for child in d['childs']:
                try:
                    category_name = child['name']
                    category_url = child['url']
                    shard = child['shard']
                    query = child['query']
                    data_list.append({
                        'category_name': category_name,
                        'category_url': category_url,
                        'shard': shard,
                        'query': query})
                except:
                    continue
                try:
                    for sub_child in child['childs']:
                        category_name = sub_child['name']
                        category_url = sub_child['url']
                        shard = sub_child['shard']
                        query = sub_child['query']
                        data_list.append({
                            'category_name': category_name,
                            'category_url': category_url,
                            'shard': shard,
                            'query': query})
                except:
                    # print(f'не имеет дочерних каталогов *{i["name"]}*')
                    continue
        except:
            # print(f'не имеет дочерних каталогов *{d["name"]}*')
            continue
    return data_list

def search_category_in_catalog(url, catalog_list):
    """пишем проверку пользовательской ссылки на наличии в каталоге"""
    try:
        for catalog in catalog_list:
            if catalog['category_url'] == url.split('https://www.wildberries.ru')[-1]:
                print(f'найдено совпадение: {catalog["category_name"]}')
                name_category = catalog['category_name']
                shard = catalog['shard']
                query = catalog['query']
                return name_category, shard, query
            else:
                # print('нет совпадения')
                pass
    except:
        print('Данный раздел не найден!')

def get_data_from_json(json_file):
    """извлекаем из json интересующие нас данные"""
    data_list = []
    for data in json_file['data']['products']:
        try:
            price = int(data["priceU"] / 100)
        except:
            price = 0
        data_list.append({
            'Наименование': data['name'],
            'id': data['id'],
            'Скидка': data['sale'],
            'Цена': price,
            'Цена со скидкой': int(data["salePriceU"] / 100),
            'Бренд': data['brand'],
            'id бренда': int(data['brandId']),
            'feedbacks': data['feedbacks'],
            'rating': data['rating'],
            'Ссылка': f'https://www.wildberries.ru/catalog/{data["id"]}/detail.aspx?targetUrl=BP'
        })
    return data_list

def get_content(shard, query, low_price=None, top_price=None):
    # вставляем ценовые рамки для уменьшения выдачи, вилбериес отдает только 100 страниц
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    data_list = []
    for page in range(1, 101):
        print(f'Сбор позиций со страницы {page} из 100')
        # url = f'https://wbxcatalog-ru.wildberries.ru/{shard}' \
        #       f'/catalog?appType=1&curr=rub&dest=-1029256,-102269,-1278703,-1255563' \
        #       f'&{query}&lang=ru&locale=ru&sort=sale&page={page}' \
        #       f'&priceU={low_price * 100};{top_price * 100}'
        url = f'https://catalog.wb.ru/catalog/{shard}/catalog?appType=1&curr=rub&dest=-1075831,-77677,-398551,12358499' \
              f'&locale=ru&page={page}&priceU={low_price * 100};{top_price * 100}' \
              f'&reg=0&regions=64,83,4,38,80,33,70,82,86,30,69,1,48,22,66,31,40&sort=popular&spp=0&{query}'
        r = requests.get(url, headers=headers)
        data = r.json()
        print(f'Добавлено позиций: {len(get_data_from_json(data))}')
        if len(get_data_from_json(data)) > 0:
            data_list.extend(get_data_from_json(data))
        else:
            print(f'Сбор данных завершен.')
            break
    return data_list

def save_excel(data, filename):
    """сохранение результата в excel файл"""
    df = pd.DataFrame(data)
    writer = pd.ExcelWriter(f'{filename}.xlsx')
    df.to_excel(writer, 'data')
    writer.save()
    print(f'Все сохранено в {filename}.xlsx')

def parser(url, low_price, top_price):
    # получаем список каталогов
    catalog_list = get_catalogs_wb()
    try:
        # поиск введенной категории в общем каталоге
        name_category, shard, query = search_category_in_catalog(url=url, catalog_list=catalog_list)
        # сбор данных в найденном каталоге
        data_list = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        # сохранение найденных данных
        save_excel(data_list, f'{name_category}_from_{low_price}_to_{top_price}')
    except TypeError:
        print('Ошибка! Возможно не верно указан раздел. Удалите все доп фильтры с ссылки')
    except PermissionError:
        print('Ошибка! Вы забыли закрыть созданный ранее excel файл. Закройте и повторите попытку')

In [ ]:
if __name__ == '__main__':
    """ссылку на каталог или подкаталог, указывать без фильтров (без ценовых, сортировки и тд.)"""
    # url = input('Введите ссылку на категорию для сбора: ')
    # low_price = int(input('Введите минимальную сумму товара: '))
    # top_price = int(input('Введите максимульную сумму товара: '))

    """данные для теста. собераем товар с раздела велосипеды в ценовой категории от 50тыс, до 100тыс"""
    url = 'https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki'
    low_price = 10_000
    top_price = 1_000_000

    parser(url, low_price, top_price)

In [146]:
file = 'C:\\Python\\Jupyter_VSC\\MIPT_DS\\1 semester\\datatone\\Ноутбуки_from_10000_to_1000000.xlsx'
df_products = pd.read_excel(file)
df_products.head(2)

,Unnamed: 0,Наименование,id,Скидка,Цена,Цена со скидкой,Бренд,id бренда,feedbacks,rating,Ссылка
0,0,"NB283 14"" N3350, 14"" 1366*768 IPS",103561157,21,17390,13738,Irbis,18909,68,5,https://www.wildberries.ru/catalog/103561157/d...
1,1,Intel Core i5 2.4ГГц 8ГБ 512ГБ,132526920,16,64990,54591,Realme,48914,9,5,https://www.wildberries.ru/catalog/132526920/d...


In [165]:
df_products.shape

(897, 11)

In [198]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#словарь для хранения информации
all_info = dict()

try:
    for index, rows in tqdm(df_products.iterrows()):
        # if index > 5: break

        url = rows['Ссылка']
        id = rows['id']
    
        driver.get(url=url)

        #даем странице время прогрузиться
        time.sleep(1.5 + 2 * random.random())

        #инициализируем словари для временного хранения информаци
        seller = dict()
        parameters = dict()

        #объект bs4 для парсинга
        soup = BeautifulSoup(driver.page_source, 'lxml')

        #вытаскиваем параметры из таблицы "О товаре"
        product_parameters = soup.body.main.find_all('div',class_="product-params")[-1].find_all('table')

        for table in product_parameters:

            temp_param = dict()
            for row in table.find_all('tr'):
                temp_param[row.th.text.strip()] = row.td.text.strip()

            parameters[table.caption.text] = temp_param
        
        #вытаскиваем информацию о продавце
        seller_info = soup.body.main.find_all('div',class_="seller-info__header")

        #для WB и прочих продавцов разная структура
        if seller_info[0].find('a',class_=re.compile('seller-info__name')) is None:
            #парсинг для WB
            seller['seller_name'] = seller_info[0].find('h3',class_="seller-info__default-name").text.strip()
            seller['seller_rating'] = -1
            seller['seller_sales'] = -1

        else:
            #парсинг для сторонних продавцов
            seller['seller_name'] = seller_info[0].find('a',class_=re.compile('seller-info__name')).text.strip()
            seller['seller_rating'] = seller_info[0].find('span',class_="address-rate-mini").text.strip()
            seller['seller_sales'] = seller_info[0].find('span',class_="seller-info__review").text.strip()

        all_info[id] = {
            'seller' : seller,
            'parameters' : parameters
        }
    
except Exception as ex:
    print(ex)

finally:
    driver.close
    driver.quit

897it [42:26,  2.84s/it]


In [201]:
with open('wb_parsed_info.json', 'w', encoding='UTF-8') as file:
    json.dump(all_info, file, indent=2, ensure_ascii=False)